# No modificar, todo funciona

In [20]:
# Librerias necesarias
import pandas as pd
import os
import json

In [21]:
# Cargar los datos
df = pd.read_excel("../2. data/0 TimeLog (original, no modificar).xlsx")

In [26]:
# Copio para no modificar el original
tl = df.copy()
# Agrego LOS a cada fila y reordeno
tl["LOS"] = (tl["TF"] - tl["TI"])


In [27]:
# Tabla para los OR
resultados = {}

tabla = {
    1: [["MS_GRD", "LOS = 12", "LOS = 24"]],
    2: [["MS_GRD", "LOS = 12", "LOS = 24"]],
    3: [["MS_GRD", "LOS = 12", "LOS = 24"]]
}

for hospital in range(1,4):
    resultados[hospital] = {}
    for grd in range(1,9):
        for unidad in ["OR"]:
            # ---- 1. Filter your dataset
            tl_u = tl[tl["UNIDAD"].isin(["ICU", "OR", "SDU_WARD"])]
            v1 = tl_u[(tl_u["UNIDAD"] == unidad) & (tl_u["MS_GRD"] == grd) & (tl_u["HOSPITAL"] == f"Hospital_{hospital}")]
            # v1 = tl_u[(tl_u["UNIDAD"] == unidad) & (tl_u["MS_GRD"] == grd)]

            # ---- 2. Get LOS value counts
            vector = v1["LOS"].value_counts().reset_index().sort_values(by="LOS")
            vector["%"] = vector["count"] / vector["count"].sum()
            vector1 = vector[["LOS", "%"]].reset_index()

            los_12 = 0
            los_24 = 0
            for index, row in vector1.iterrows():
                if row["LOS"] == 12:
                    los_12 = row["%"]
                elif row["LOS"] == 24:
                    los_24 = row["%"]

            tabla[hospital].append([grd, round(los_12,5), round(los_24,5)])
            resultados[hospital][grd] = {"final_kde_pmf": [los_12, los_24]}
                    
            

    display(tabla[hospital])

[['MS_GRD', 'LOS = 12', 'LOS = 24'],
 [1, 0.82487, 0.17513],
 [2, 0.78787, 0.21213],
 [3, 0.8561, 0.1439],
 [4, 0.84994, 0.15006],
 [5, 0.99358, 0.00642],
 [6, 1.0, 0],
 [7, 0.89594, 0.10406],
 [8, 0.90576, 0.09424]]

[['MS_GRD', 'LOS = 12', 'LOS = 24'],
 [1, 0.81692, 0.18308],
 [2, 0.78138, 0.21862],
 [3, 0.86486, 0.13514],
 [4, 0.84202, 0.15798],
 [5, 0.99008, 0.00992],
 [6, 1.0, 0],
 [7, 0.91046, 0.08954],
 [8, 0.90485, 0.09515]]

[['MS_GRD', 'LOS = 12', 'LOS = 24'],
 [1, 0.81719, 0.18281],
 [2, 0.77562, 0.22438],
 [3, 0.87247, 0.12753],
 [4, 0.84805, 0.15195],
 [5, 0.99125, 0.00875],
 [6, 1.0, 0],
 [7, 0.90361, 0.09639],
 [8, 0.91713, 0.08287]]

In [28]:
def save_dict_as_json(data_dict, filename, folder):
    os.makedirs(folder, exist_ok=True)  # Create folder if it doesn't exist
    path = os.path.join(folder, filename)
    with open(path, 'w') as f:
        json.dump(data_dict, f, indent=4)
    print(f"Dictionary saved to: {path}")

# Save the results to a JSON file
save_dict_as_json(resultados, filename="los_OR.json", folder="resultados incertidumbre")

Dictionary saved to: resultados incertidumbre/los_OR.json


In [29]:
# Pasar a tablas de latex
for key, h in tabla.items():

    texto = f"""
\\begin{{table}}[H]
    \\centering
    \\begin{{tabular}}{{ccc}}
        \\toprule
        MS\_GRD & LOS = 12 & LOS = 24 \\\\
        \\midrule
        {h[1][0]} & {h[1][1]} & {h[1][2]} \\\\
        {h[2][0]} & {h[2][1]} & {h[2][2]} \\\\
        {h[3][0]} & {h[3][1]} & {h[3][2]} \\\\
        {h[4][0]} & {h[4][1]} & {h[4][2]} \\\\
        {h[5][0]} & {h[5][1]} & {h[5][2]} \\\\
        {h[6][0]} & {h[6][1]} & {h[6][2]} \\\\
        {h[7][0]} & {h[7][1]} & {h[7][2]} \\\\
        {h[8][0]} & {h[8][1]} & {h[8][2]} \\\\
        \\bottomrule
    \\end{{tabular}}
    \\caption{{Probabilidad LOS en OR para Hospital: {key}}}
    \\label{{tab:Probabilidad LOS en OR para Hospital: {key}}}
\\end{{table}}
    """

    print(texto)


\begin{table}[H]
    \centering
    \begin{tabular}{ccc}
        \toprule
        MS\_GRD & LOS = 12 & LOS = 24 \\
        \midrule
        1 & 0.82487 & 0.17513 \\
        2 & 0.78787 & 0.21213 \\
        3 & 0.8561 & 0.1439 \\
        4 & 0.84994 & 0.15006 \\
        5 & 0.99358 & 0.00642 \\
        6 & 1.0 & 0 \\
        7 & 0.89594 & 0.10406 \\
        8 & 0.90576 & 0.09424 \\
        \bottomrule
    \end{tabular}
    \caption{Probabilidad LOS en OR para Hospital: 1}
    \label{tab:Probabilidad LOS en OR para Hospital: 1}
\end{table}
    

\begin{table}[H]
    \centering
    \begin{tabular}{ccc}
        \toprule
        MS\_GRD & LOS = 12 & LOS = 24 \\
        \midrule
        1 & 0.81692 & 0.18308 \\
        2 & 0.78138 & 0.21862 \\
        3 & 0.86486 & 0.13514 \\
        4 & 0.84202 & 0.15798 \\
        5 & 0.99008 & 0.00992 \\
        6 & 1.0 & 0 \\
        7 & 0.91046 & 0.08954 \\
        8 & 0.90485 & 0.09515 \\
        \bottomrule
    \end{tabular}
    \caption{Probabilidad